# Importing The Necessary Dependencies

In [68]:
!pip install tensorflow==2.12.0

!pip install keras_cv==0.5.0
!pip install keras-flops
import tensorflow as tf

# Check TensorFlow and Keras version
tensorflow_version = tf.__version__
keras_version = tf.keras.__version__

print(f"TensorFlow Version: {tensorflow_version}")
print(f"Keras Version: {keras_version}")

from keras import backend as K

TensorFlow Version: 2.12.0
Keras Version: 2.12.0


In [69]:
import os
from PIL import Image
import pandas as pd

# Target path
image_dir = "/kaggle/working/stare_aug/train/images"

# Store image filenames and resolutions
image_sizes = []

# Iterate through all image files in the folder
for filename in sorted(os.listdir(image_dir)):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        path = os.path.join(image_dir, filename)
        try:
            with Image.open(path) as img:
                image_sizes.append((filename, img.size))  # (width, height)
        except Exception as e:
            image_sizes.append((filename, f"Error: {str(e)}"))

# Convert to DataFrame for easier viewing
df = pd.DataFrame(image_sizes, columns=["Filename", "Resolution"])

# Display the first few rows
print(df.head())

# Save to CSV if needed
# df.to_csv("/kaggle/working/image_resolutions.csv", index=False)


                   Filename  Resolution
0  diagonalFlip0_im0001.png  (605, 700)
1  diagonalFlip0_im0002.png  (605, 700)
2  diagonalFlip0_im0003.png  (605, 700)
3  diagonalFlip0_im0004.png  (605, 700)
4  diagonalFlip0_im0005.png  (605, 700)


In [70]:
def conv_block(x, filters, groups=8, block_size=7, drop_rate=0.15):
    """
    Conv2D → DropBlock2D → GroupNorm → SiLU
    """
    # ------- Conv layer 1 -------
    x = Conv2D(filters, (3, 3), padding="same",
               activation=None, kernel_initializer="he_normal")(x)
    x = DropBlock2D(block_size=block_size, rate=drop_rate)(x)
    x = GroupNormalization(groups=groups, axis=-1)(x)
    x = Activation('silu')(x)

    # ------- Conv layer 2 -------
    x = Conv2D(filters, (3, 3), padding="same",
               activation=None, kernel_initializer="he_normal")(x)
    x = DropBlock2D(block_size=block_size, rate=drop_rate)(x)
    x = GroupNormalization(groups=groups, axis=-1)(x)
    x = Activation('silu')(x)

    return x

In [71]:
from keras.models import Model
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, Activation, concatenate
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU

def UNet(input_size=(592, 592, 3),start_neurons=16):
    inputs = Input(input_size)

    # Encoder
    conv1 = conv_block(inputs, start_neurons * 1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = conv_block(pool1, start_neurons * 2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = conv_block(pool2, start_neurons * 4)
    pool3 = MaxPooling2D((2, 2))(conv3)

    conv4 = conv_block(pool3, start_neurons * 8)
    pool4 = MaxPooling2D((2, 2))(conv4)

    # Bottleneck
    convm = conv_block(pool4, start_neurons * 16)

    # Decoder
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = conv_block(uconv4, start_neurons * 8)

    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    uconv3 = concatenate([deconv3, conv3])
    uconv3 = conv_block(uconv3, start_neurons * 4)

    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = conv_block(uconv2, start_neurons * 2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = conv_block(uconv1, start_neurons * 1)

    output_layer_noActi = Conv2D(1, (1, 1), padding="same", activation=None, kernel_initializer='he_normal')(uconv1)
    output_layer = Activation('sigmoid')(output_layer_noActi)

    model = Model(inputs, output_layer)
    return model




In [72]:
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, multiply, Permute, Concatenate, \
    Conv2D, Add, Activation, Lambda,Conv1D

def spatial_attention(input_feature):
    kernel_size = 7

    if K.image_data_format() == "channels_first":
        channel = input_feature.shape[1]
        cbam_feature = Permute((2, 3, 1))(input_feature)
    else:
        channel = input_feature.shape[-1]
        cbam_feature = input_feature

    avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
    assert avg_pool.shape[-1] == 1
    max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
    assert max_pool.shape[-1] == 1
    concat = Concatenate(axis=3)([avg_pool, max_pool])
    assert concat.shape[-1] == 2
    cbam_feature = Conv2D(filters=1,
                          kernel_size=kernel_size,
                          strides=1,
                          padding='same',
                          activation='sigmoid',
                          kernel_initializer='he_normal',
                          use_bias=False)(concat)
    assert cbam_feature.shape[-1] == 1

    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)

    return multiply([input_feature, cbam_feature])

In [73]:
def CSA(concat1, concat2, kernel_size=7):

    if K.image_data_format() == "channels_first":
        channel1 = concat1.shape[1]
        sa1 = Permute((2, 3, 1))(concat1)
        channel2 = concat2.shape[1]
        sa2 = Permute((2, 3, 1))(concat2)
    else:
        channel1 = concat1.shape[-1]
        sa1 = concat1
        channel2 = concat2.shape[-1]
        sa2 = concat2

    avg_pool1 = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(sa1)
    assert avg_pool1.shape[-1] == 1
    avg_pool2 = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(sa2)
    assert avg_pool2.shape[-1] == 1

    sa = Concatenate(axis=3)([avg_pool1, avg_pool2])
    assert sa.shape[-1] == 2

    sa = Conv2D(filters=1,
                kernel_size=kernel_size,
                strides=1,
                padding='same',
                activation='sigmoid',
                kernel_initializer='he_normal',
                use_bias=False)(sa)  
    assert sa.shape[-1] == 1

    if K.image_data_format() == "channels_first":
        sa = Permute((3, 1, 2))(sa)

    return multiply([concat1, sa])

In [74]:

from keras.models import Model
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, Activation, concatenate
from keras_cv.layers import DropBlock2D
from tensorflow.keras.layers import GroupNormalization,BatchNormalization

def SA_UNet(input_size=(1008, 1008, 3), block_size=7,rate=0.1,start_neurons=16,lr=1e-3):

    inputs = Input(input_size)
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(inputs)
    conv1 = DropBlock2D(block_size=block_size, rate=rate)(conv1)
    conv1= BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(conv1)
    conv1 = DropBlock2D(block_size=block_size, rate=rate)(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(pool1)
    conv2 = DropBlock2D(block_size=block_size, rate=rate)(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)

    conv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(conv2)
    conv2 = DropBlock2D(block_size=block_size, rate=rate)(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)


    conv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(pool2)
    conv3 = DropBlock2D(block_size=block_size, rate=rate)(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(conv3)
    conv3 = DropBlock2D(block_size=block_size, rate=rate)(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)


    convm = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(pool3)
    convm = DropBlock2D(block_size=block_size, rate=rate)(convm)
    convm = BatchNormalization()(convm)
    convm = Activation('relu')(convm)
    convm = spatial_attention(convm)
    convm = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(convm)
    convm = DropBlock2D(block_size=block_size, rate=rate)(convm)
    convm = BatchNormalization()(convm)
    convm = Activation('relu')(convm)


    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv3 = concatenate([deconv3, conv3])

    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv3)
    uconv3 = DropBlock2D(block_size=block_size, rate=rate)(uconv3)
    uconv3 = BatchNormalization()(uconv3)
    uconv3 = Activation('relu')(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv3)
    uconv3 = DropBlock2D(block_size=block_size, rate=rate)(uconv3)
    uconv3 = BatchNormalization()(uconv3)
    uconv3 = Activation('relu')(uconv3)

    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])

    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv2)
    uconv2 = DropBlock2D(block_size=block_size, rate=rate)(uconv2)
    uconv2 = BatchNormalization()(uconv2)
    uconv2 = Activation('relu')(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv2)
    uconv2 = DropBlock2D(block_size=block_size, rate=rate)(uconv2)
    uconv2 = BatchNormalization()(uconv2)
    uconv2 = Activation('relu')(uconv2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])


    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv1)
    uconv1 = DropBlock2D(block_size=block_size, rate=rate)(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    uconv1 = Activation('relu')(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv1)
    uconv1 = DropBlock2D(block_size=block_size, rate=rate)(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    uconv1 = Activation('relu')(uconv1)
    output_layer_noActi = Conv2D(1, (1, 1), padding="same", activation=None)(uconv1)
    output_layer = Activation('sigmoid')(output_layer_noActi)
    model = Model(inputs, output_layer)

    return model


In [75]:
!pip install keras-flops

In [76]:
from keras.models import Model
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, Activation, concatenate
from keras_cv.layers import DropBlock2D
from tensorflow_addons.layers import GroupNormalization

def SA_UNetGN_SiLU(input_size=(592, 592, 3), block_size=7, rate=0.1, start_neurons=16):

    inputs = Input(input_size)

    conv1 = Conv2D(start_neurons * 1, (3, 3), padding="same", activation=None, kernel_initializer='he_normal')(inputs)
    conv1 = DropBlock2D(block_size=block_size, rate=rate)(conv1)
    conv1 = GroupNormalization(groups=8)(conv1)
    conv1 = Activation('silu')(conv1)
    conv1 = Conv2D(start_neurons * 1, (3, 3), padding="same", activation=None, kernel_initializer='he_normal')(conv1)
    conv1 = DropBlock2D(block_size=block_size, rate=rate)(conv1)
    conv1 = GroupNormalization(groups=8)(conv1)
    conv1 = Activation('silu')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(start_neurons * 2, (3, 3), padding="same", activation=None, kernel_initializer='he_normal')(pool1)
    conv2 = DropBlock2D(block_size=block_size, rate=rate)(conv2)
    conv2 = GroupNormalization(groups=8)(conv2)
    conv2 = Activation('silu')(conv2)
    conv2 = Conv2D(start_neurons * 2, (3, 3), padding="same", activation=None, kernel_initializer='he_normal')(conv2)
    conv2 = DropBlock2D(block_size=block_size, rate=rate)(conv2)
    conv2 = GroupNormalization(groups=8)(conv2)
    conv2 = Activation('silu')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(start_neurons * 4, (3, 3), padding="same", activation=None, kernel_initializer='he_normal')(pool2)
    conv3 = DropBlock2D(block_size=block_size, rate=rate)(conv3)
    conv3 = GroupNormalization(groups=8)(conv3)
    conv3 = Activation('silu')(conv3)
    conv3 = Conv2D(start_neurons * 4, (3, 3), padding="same", activation=None, kernel_initializer='he_normal')(conv3)
    conv3 = DropBlock2D(block_size=block_size, rate=rate)(conv3)
    conv3 = GroupNormalization(groups=8)(conv3)
    conv3 = Activation('silu')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    convm = Conv2D(start_neurons * 8, (3, 3), padding="same", activation=None, kernel_initializer='he_normal')(pool3)
    convm = DropBlock2D(block_size=block_size, rate=rate)(convm)
    convm = GroupNormalization(groups=8)(convm)
    convm = Activation('silu')(convm)
    convm = spatial_attention(convm)
    convm = Conv2D(start_neurons * 8, (3, 3), padding="same", activation=None, kernel_initializer='he_normal')(convm)
    convm = DropBlock2D(block_size=block_size, rate=rate)(convm)
    convm = GroupNormalization(groups=8)(convm)
    convm = Activation('silu')(convm)

    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same", kernel_initializer='he_normal')(convm)
    uconv3 = concatenate([deconv3, conv3])
    uconv3 = Conv2D(start_neurons * 4, (3, 3), padding="same", activation=None, kernel_initializer='he_normal')(uconv3)
    uconv3 = DropBlock2D(block_size=block_size, rate=rate)(uconv3)
    uconv3 = GroupNormalization(groups=8)(uconv3)
    uconv3 = Activation('silu')(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), padding="same", activation=None, kernel_initializer='he_normal')(uconv3)
    uconv3 = DropBlock2D(block_size=block_size, rate=rate)(uconv3)
    uconv3 = GroupNormalization(groups=8)(uconv3)
    uconv3 = Activation('silu')(uconv3)

    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same", kernel_initializer='he_normal')(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Conv2D(start_neurons * 2, (3, 3), padding="same", activation=None, kernel_initializer='he_normal')(uconv2)
    uconv2 = DropBlock2D(block_size=block_size, rate=rate)(uconv2)
    uconv2 = GroupNormalization(groups=8)(uconv2)
    uconv2 = Activation('silu')(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), padding="same", activation=None, kernel_initializer='he_normal')(uconv2)
    uconv2 = DropBlock2D(block_size=block_size, rate=rate)(uconv2)
    uconv2 = GroupNormalization(groups=8)(uconv2)
    uconv2 = Activation('silu')(uconv2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same", kernel_initializer='he_normal')(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Conv2D(start_neurons * 1, (3, 3), padding="same", activation=None, kernel_initializer='he_normal')(uconv1)
    uconv1 = DropBlock2D(block_size=block_size, rate=rate)(uconv1)
    uconv1 = GroupNormalization(groups=8)(uconv1)
    uconv1 = Activation('silu')(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), padding="same", activation=None, kernel_initializer='he_normal')(uconv1)
    uconv1 = DropBlock2D(block_size=block_size, rate=rate)(uconv1)
    uconv1 = GroupNormalization(groups=8)(uconv1)
    uconv1 = Activation('silu')(uconv1)

    output_layer_noActi = Conv2D(1, (1, 1), padding="same", activation=None, kernel_initializer='he_normal')(uconv1)
    output_layer = Activation('sigmoid')(output_layer_noActi)

    model = Model(inputs, output_layer)
    return model


In [77]:
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, concatenate, Lambda
from keras_cv.layers import DropBlock2D
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf


def SA_UNetV2(input_size=(1008, 1008, 3), block_size=7, rate=0.1, start_neurons=16, lr=1e-3):
    inputs = Input(input_size)

    conv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same",kernel_initializer='he_normal')(inputs)
    conv1 = DropBlock2D(block_size=block_size, rate=rate)(conv1)
    conv1 = GroupNormalization(groups=8)(conv1)
    conv1 = Activation('silu')(conv1)
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same",kernel_initializer='he_normal')(conv1)
    conv1 = DropBlock2D(block_size=block_size, rate=rate)(conv1)
    conv1 = GroupNormalization(groups=8)(conv1)
    conv1 = Activation('silu')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same",kernel_initializer='he_normal')(pool1)
    conv2 = DropBlock2D(block_size=block_size, rate=rate)(conv2)
    conv2 = GroupNormalization(groups=8)(conv2)
    conv2 = Activation('silu')(conv2)
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same",kernel_initializer='he_normal')(conv2)
    conv2 = DropBlock2D(block_size=block_size, rate=rate)(conv2)
    conv2 = GroupNormalization(groups=8)(conv2)
    conv2 = Activation('silu')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(start_neurons * 3, (3, 3), activation=None, padding="same",kernel_initializer='he_normal')(pool2)
    conv3 = DropBlock2D(block_size=block_size, rate=rate)(conv3)
    conv3 = GroupNormalization(groups=8)(conv3)
    conv3 = Activation('silu')(conv3)
    conv3 = Conv2D(start_neurons * 3, (3, 3), activation=None, padding="same",kernel_initializer='he_normal')(conv3)
    conv3 = DropBlock2D(block_size=block_size, rate=rate)(conv3)
    conv3 = GroupNormalization(groups=8)(conv3)
    conv3 = Activation('silu')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)


    convm = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same",kernel_initializer='he_normal')(pool3)
    convm = DropBlock2D(block_size=block_size, rate=rate)(convm)
    convm = GroupNormalization(groups=8)(convm)
    convm = Activation('silu')(convm)
    convm = spatial_attention(convm)
    convm = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same",kernel_initializer='he_normal')(convm)
    convm = DropBlock2D(block_size=block_size, rate=rate)(convm)
    convm = GroupNormalization(groups=8)(convm)
    convm = Activation('silu')(convm)

    deconv3 = Conv2DTranspose(start_neurons * 3, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv3 = concatenate([deconv3, CSA(conv3,deconv3)])
    uconv3 = Conv2D(start_neurons * 3, (3, 3), activation=None, padding="same",kernel_initializer='he_normal')(uconv3)
    uconv3 = DropBlock2D(block_size=block_size, rate=rate)(uconv3)
    uconv3 = GroupNormalization(groups=8)(uconv3)
    uconv3 = Activation('silu')(uconv3)
    uconv3 = Conv2D(start_neurons * 3, (3, 3), activation=None, padding="same",kernel_initializer='he_normal')(uconv3)
    uconv3 = DropBlock2D(block_size=block_size, rate=rate)(uconv3)
    uconv3 = GroupNormalization(groups=8)(uconv3)
    uconv3 = Activation('silu')(uconv3)

    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, CSA(conv2,deconv2)])
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same",kernel_initializer='he_normal')(uconv2)
    uconv2 = DropBlock2D(block_size=block_size, rate=rate)(uconv2)
    uconv2 = GroupNormalization(groups=8)(uconv2)
    uconv2 = Activation('silu')(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same",kernel_initializer='he_normal')(uconv2)
    uconv2 = DropBlock2D(block_size=block_size, rate=rate)(uconv2)
    uconv2 = GroupNormalization(groups=8)(uconv2)
    uconv2 = Activation('silu')(uconv2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, CSA(conv1,deconv1)])
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same",kernel_initializer='he_normal')(uconv1)
    uconv1 = DropBlock2D(block_size=block_size, rate=rate)(uconv1)
    uconv1 = GroupNormalization(groups=8)(uconv1)
    uconv1 = Activation('silu')(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same",kernel_initializer='he_normal')(uconv1)
    uconv1 = DropBlock2D(block_size=block_size, rate=rate)(uconv1)
    uconv1 = GroupNormalization(groups=8)(uconv1)
    uconv1 = Activation('silu')(uconv1)

    output_layer_noActi = Conv2D(1, (1, 1), padding="same", activation=None,kernel_initializer='he_normal')(uconv1)
    output_layer = Activation('sigmoid')(output_layer_noActi)

    model = Model(inputs, output_layer)

    return model


In [78]:
import os
import numpy as np
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split

def get_max_shape(image_dir):
    max_h, max_w = 0, 0
    for file in sorted(os.listdir(image_dir)):
        if file.endswith('.png'):
            img = Image.open(os.path.join(image_dir, file))
            h, w = img.size[1], img.size[0]
            max_h = max(max_h, h)
            max_w = max(max_w, w)
    return max_h, max_w

def round_up_to_multiple(value, multiple):
    return ((value + multiple - 1) // multiple) * multiple

def pad_images(img, target_shape):
    h, w = img.shape[:2]
    target_h, target_w = target_shape

    pad_h = max(target_h - h, 0)
    pad_w = max(target_w - w, 0)
    pad_top = pad_h // 2
    pad_bottom = pad_h - pad_top
    pad_left = pad_w // 2
    pad_right = pad_w - pad_left

    if img.ndim == 3:
        return np.pad(img, ((pad_top, pad_bottom), (pad_left, pad_right), (0, 0)), mode='constant')
    else:
        return np.pad(img, ((pad_top, pad_bottom), (pad_left, pad_right)), mode='constant')

def load_stare_data(image_dir, label_dir, target_shape):
    image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.png')])
    x_data, y_data = [], []

    for img_name in image_files:
        base_name = os.path.splitext(img_name)[0]
        label_name = f"{base_name}.ah.png"

        img_path = os.path.join(image_dir, img_name)
        label_path = os.path.join(label_dir, label_name)

        if not os.path.exists(label_path):
            print(f"⚠️ Missing label: {label_path}, skipping")
            continue

        img = np.array(Image.open(img_path).convert('RGB'))
        label = np.array(Image.open(label_path).convert('L'))

        img_padded = pad_images(img, target_shape)
        label_padded = pad_images(label, target_shape)

        _, label_bin = cv2.threshold(label_padded, 127, 255, cv2.THRESH_BINARY)
        label_bin = np.expand_dims(label_bin, axis=-1)

        x_data.append(img_padded)
        y_data.append(label_bin)

    x_data = np.array(x_data, dtype=np.float32) / 255.0
    y_data = np.array(y_data, dtype=np.float32) / 255.0
    return x_data, y_data

# ==== ✅ Main Entry Point ====
train_image_dir = '/kaggle/working/stare_aug/train/images'
train_label_dir = '/kaggle/working/stare_aug/train/labels'

# Step 1️⃣ Get max shape from training set
train_max_h, train_max_w = get_max_shape(train_image_dir)
target_h = round_up_to_multiple(train_max_h, 8)
target_w = round_up_to_multiple(train_max_w, 8)
target_shape = (target_h, target_w)

print(f"📐 Max train image size: ({train_max_h}, {train_max_w}) → Padded to: {target_shape}")

# Step 2️⃣ Load and pad images
x_all, y_all = load_stare_data(train_image_dir, train_label_dir, target_shape)

# Step 3️⃣ Split train/val (90/10)
x_train, x_val, y_train, y_val = train_test_split(
    x_all, y_all, test_size=0.1, shuffle=True, random_state=42
)

print('✅ x_train shape:', x_train.shape)
print('✅ y_train shape:', y_train.shape)
print('✅ x_val shape:', x_val.shape)
print('✅ y_val shape:', y_val.shape)


📐 Max train image size: (700, 700) → Padded to: (704, 704)
✅ x_train shape: (187, 704, 704, 3)
✅ y_train shape: (187, 704, 704, 1)
✅ x_val shape: (21, 704, 704, 3)
✅ y_val shape: (21, 704, 704, 1)


In [79]:
import tensorflow as tf
from skimage.morphology import skeletonize
import numpy as np
from tensorflow import keras
import tensorflow.keras.backend as K


def dice_loss(y_true, y_pred, smooth=1e-6):
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    loss = 1 - (2. * intersection + smooth) / (union + smooth)
    return loss

def jaccard_loss(y_true, y_pred, smooth=1e-6):
    """Jaccard Loss (Intersection over Union) for binary segmentation."""
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    loss = 1 - (intersection + smooth) / (union + smooth)
    return loss

def ssim_loss(y_true, y_pred):
    ssim = tf.image.ssim(y_true, y_pred, max_val=1.0)
    return 1 - tf.reduce_mean(ssim)

def gradient_regularizer(y_pred):
    dx = tf.abs(y_pred[:,1:,:,:] - y_pred[:,:-1,:,:])
    dy = tf.abs(y_pred[:,:,1:,:] - y_pred[:,:,:-1,:])
    return tf.reduce_mean(dx) + tf.reduce_mean(dy)



class MCC_Loss(keras.losses.Loss):
    def __init__(self, name="mcc_loss"):
        super(MCC_Loss, self).__init__(name=name)

    def call(self, y_true, y_pred):
        """
        Calculates the Matthews Correlation Coefficient (MCC) loss.
        
        Arguments:
            y_true: Ground truth labels, shape (batch_size, num_classes).
            y_pred: Model predictions (probabilities), shape (batch_size, num_classes).
            
        Returns:
            A scalar value representing the MCC loss.
        """
        # Keep the predictions as continuous probabilities instead of applying a hard threshold
        tp = K.sum(y_pred * y_true)  # True Positives
        tn = K.sum((1 - y_true) * (1 - y_pred))  # True Negatives
        fp = K.sum((1 - y_true) * y_pred)  # False Positives
        fn = K.sum(y_true * (1 - y_pred))  # False Negatives
        
        # Compute MCC using the formula
        numerator = tp * tn - fp * fn
        denominator = K.sqrt(
            (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
        )
        
        # MCC loss: 1 - MCC
        mcc = numerator / (denominator + K.epsilon())  # Adding epsilon to avoid divide by zero
        return 1 - mcc  # We return 1 - MCC because we are minimizing the loss

def combined_loss(y_true, y_pred):
    bce = keras.losses.binary_crossentropy(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    jaccard = jaccard_loss(y_true, y_pred)  # 新增的Jaccard Loss
    ssim = ssim_loss(y_true, y_pred)
    smoothness = gradient_regularizer(y_pred)
    mcc = MCC_Loss()(y_true, y_pred) 
    
    return 0.5 * mcc + 0.5 * bce

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau  # newly added import
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

# Create model
model_name = "SAUNetV2"

if model_name == "SAUNet":
    model = SA_UNetGN_SiLU(
        input_size=(704, 704, 3),
        start_neurons=16,
        rate=0.15,
        block_size=7
    )

elif model_name == "SAUNetV2":
    model = SA_UNetV2(
        input_size=(704, 704, 3),
        start_neurons=16,
        rate=0.15,
        block_size=7
    )

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    # loss="binary_crossentropy", # use defined combined loss
    loss=combined_loss,
    metrics=['accuracy']
)

model.summary()
weight = "SA_UNetv2.h5"
restore = True

# Load model weights if available
if restore and os.path.isfile(weight):
    model.load_weights(weight)

# Configure callbacks
callbacks = [
    # Save best weights (monitoring validation accuracy)
    ModelCheckpoint(
        weight,
        monitor='val_accuracy',  # you can also use 'val_loss'
        verbose=1,
        save_best_only=True,
        save_weights_only=True,
        mode='max'  # use max when monitoring accuracy, min when monitoring loss
    ),
    
    # Dynamic learning rate reduction (monitoring validation loss)
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,       # learning rate reduction factor
        patience=20,      # wait for 20 epochs without improvement
        min_lr=1e-8,      # minimum learning rate
        verbose=1
    ),
    
    # Early stopping mechanism (monitoring validation loss)
    EarlyStopping(
        monitor='val_loss',
        patience=30,      # stop after 30 epochs without improvement
        restore_best_weights=True,  # restore the best weights
        verbose=1
    )
]

# Start training
history = model.fit(
    x_train, y_train,
    epochs=150,
    batch_size=2,
    validation_data=(x_val, y_val),
    # validation_split=0.1, 
    shuffle=True,
    callbacks=callbacks
)


Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 704, 704, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_152 (Conv2D)            (None, 704, 704, 16  448         ['input_9[0][0]']                
                                )                                                                 
                                                                                                  
 drop_block2d_112 (DropBlock2D)  (None, 704, 704, 16  0          ['conv2d_152[0][0]']             
                                )                                                           

In [ ]:
import os
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

# Paths to fold4 val set
testing_images_loc = '/kaggle/working/stare/test/images'
testing_label_loc = '/kaggle/working/stare/test/labels'

# Only read .png images
test_files = sorted([f for f in os.listdir(testing_images_loc) if f.endswith('.png')])
test_data = []
test_label = []

target_size = None  # Will store the size of the first image (W, H)

for fname in test_files:
    # Construct label filename using .ah.png convention
    label_name = fname.replace('.png', '.ah.png')
    img_path = os.path.join(testing_images_loc, fname)
    label_path = os.path.join(testing_label_loc, label_name)

    if not os.path.exists(label_path):
        print(f"❌ Missing label file: {label_path}, skipping")
        continue

    # Load image and label
    im = np.array(Image.open(img_path).convert('RGB'))
    label = np.array(Image.open(label_path).convert('L'))

    # Save first image size
    if target_size is None:
        target_size = (im.shape[1], im.shape[0])  # (W, H)

    # Resize if needed
    if (im.shape[1], im.shape[0]) != target_size:
        im = cv2.resize(im, target_size)
    if (label.shape[1], label.shape[0]) != target_size:
        label = cv2.resize(label, target_size)

    # Binarize label
    _, label_bin = cv2.threshold(label, 127, 255, cv2.THRESH_BINARY)
    label_bin = np.expand_dims(label_bin, axis=-1)

    test_data.append(im)
    test_label.append(label_bin)

# Convert to NumPy and normalize
x_test = np.array(test_data, dtype=np.float32) / 255.
y_test = np.array(test_label, dtype=np.float32) / 255.

print('✅ x_test shape:', x_test.shape)
print('✅ y_test shape:', y_test.shape)

# Visualize first test label
plt.imshow(y_test[0].squeeze(), cmap='gray')
plt.axis('off')
plt.title('Sample y_test')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def pad_images(input_data, target_shape):
    padded_data = np.zeros(target_shape)
    for i in range(input_data.shape[0]):
        # Copy the original image into the padded image
        padded_data[i, :input_data.shape[1], :input_data.shape[2], :] = input_data[i]
    return padded_data

def restore_images(output_data, original_shape):
    restored_data = np.zeros(original_shape)
    for i in range(original_shape[0]):
        # Crop the padded image back to the original size
        restored_data[i] = output_data[i, :original_shape[1], :original_shape[2], :]
    return restored_data

# Test data
input_data = np.random.rand(20, 584, 565, 1)
target_shape = (20, 592, 592, 1)

# Pad the data
padded_data = pad_images(input_data, target_shape)

# Restore to original size
restored_data = restore_images(padded_data, input_data.shape)

# Verify whether the original data and restored data are identical
print(np.array_equal(input_data, restored_data))


In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

# Set a uniform input size
TARGET_SIZE = (704, 704)

# List to store original shapes
original_shapes = []

# List to store padded images
padded_test_data = []

# Automatically get original sizes and pad images
for i in range(x_test.shape[0]):
    img = x_test[i]
    h, w = img.shape[:2]
    original_shapes.append((h, w))
    
    # Create a blank image of the target size
    padded_img = np.zeros((*TARGET_SIZE, 3), dtype=np.float32)
    padded_img[:h, :w, :] = img  # Place original image
    padded_test_data.append(padded_img)

x_test_padded = np.array(padded_test_data)
print("✅ padded x_test shape:", x_test_padded.shape)

# Model prediction
import time
import numpy as np
import tensorflow as tf

# Assume model is loaded/trained
# Assume x_test_padded is the test input, shape (N, H, W, C)

# Warm-up: run once to avoid first-run delay
_ = model.predict(x_test_padded[:1])

# Timing the prediction
start = time.perf_counter()          # High-precision timer
y_pred_padded = model.predict(x_test_padded)
elapsed = time.perf_counter() - start

print("✅ y_pred padded shape:", y_pred_padded.shape)
print(f"⏱️ Prediction time: {elapsed:.4f} seconds")
print(f"⚡ Avg per sample: {elapsed / len(x_test_padded):.4f} seconds")

# Restore predictions to original sizes
y_pred_restored = []

for i in range(y_pred_padded.shape[0]):
    h, w = original_shapes[i]
    restored = y_pred_padded[i, :h, :w, :]
    y_pred_restored.append(restored)

y_pred = np.array(y_pred_restored)
print("✅ y_pred restored shape:", y_pred.shape)

# Display the 3rd predicted image (binarized)
binary_pred = (y_pred[3, ..., 0] > 0.5).astype('float32')
plt.imshow(binary_pred, cmap='gray')
plt.axis('off')
plt.title("Binary Prediction - Sample 3")
plt.show()


In [ ]:
import os

# Folder to save results
save_dir = '/kaggle/working/SA_UNetv2/'  
os.makedirs(save_dir, exist_ok=True)

from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, roc_auc_score, matthews_corrcoef, f1_score, jaccard_score
import numpy as np
import cv2

# Lists to store results
y_pred_threshold = []
accuracy_scores = []
sensitivity_scores = []
specificity_scores = []
auc_scores = []
mcc_scores = []
f1_scores = []
jaccard_scores = []

i = 0
for y_pred_image, y_test_image in zip(y_pred, y_test):
    # Binarize predictions
    _, temp = cv2.threshold(y_pred_image, 0.5, 1, cv2.THRESH_BINARY)
    y_pred_threshold.append(temp)

    # Flatten arrays for metric calculations
    y_pred_flat = np.ravel(temp)
    y_test_flat = np.ravel(y_test_image)

    # Compute confusion matrix and metrics
    tn, fp, fn, tp = confusion_matrix(y_test_flat, y_pred_flat).ravel()
    accuracy = accuracy_score(y_test_flat, y_pred_flat)
    sensitivity = recall_score(y_test_flat, y_pred_flat)
    specificity = tn / (tn + fp)
    auc = roc_auc_score(y_test_flat, np.ravel(y_pred_image))
    mcc = matthews_corrcoef(y_test_flat, y_pred_flat)
    f1 = f1_score(y_test_flat, y_pred_flat)
    jaccard = jaccard_score(y_test_flat, y_pred_flat)

    # Append metrics to lists
    accuracy_scores.append(accuracy)
    sensitivity_scores.append(sensitivity)
    specificity_scores.append(specificity)
    auc_scores.append(auc)
    mcc_scores.append(mcc)
    f1_scores.append(f1)
    jaccard_scores.append(jaccard)

    # Save probability map (0~255)
    prob_img = (y_pred_image * 255).astype(np.uint8)
    cv2.imwrite(os.path.join(save_dir, f'{i}_prob.png'), prob_img)

    # Save binary image (0 or 255)
    bin_img = (temp * 255).astype(np.uint8)
    cv2.imwrite(os.path.join(save_dir, f'{i}_bin.png'), bin_img)

    i += 1

# Calculate average metrics
avg_accuracy = np.mean(accuracy_scores)
avg_sensitivity = np.mean(sensitivity_scores)
avg_specificity = np.mean(specificity_scores)
avg_auc = np.mean(auc_scores)
avg_mcc = np.mean(mcc_scores)
avg_f1 = np.mean(f1_scores)
avg_jaccard = np.mean(jaccard_scores)

# Print average metrics
print('Average Accuracy:', avg_accuracy)
print('Average Sensitivity:', avg_sensitivity)
print('Average Specificity:', avg_specificity)
print('Average AUC:', avg_auc)
print('Average MCC:', avg_mcc)
print('Average F1 Score:', avg_f1)
print('Average Jaccard Index:', avg_jaccard)
